#Draft


In [ ]:
from datasets import load_dataset

ds = load_dataset("phantatbach/TMed")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


metadata.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/22183 [00:00<?, ? examples/s]

In [ ]:
print(ds)
print(ds["train"].features)
print(ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['file_name', 'transcription'],
        num_rows: 22183
    })
})
{'file_name': Value('string'), 'transcription': Value('string')}
{'file_name': 'data/train/225033.0000_225800.0000.ogg', 'transcription': 'So insensitive'}


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("xiaoweixumedicalai/cardiacudc-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cardiacudc-dataset


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shoybhasan/camus-human-heart-data")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/camus-human-heart-data


In [ ]:
!file /kaggle/input/camus-human-heart-data/download
!ls
!mv download camus.zip
!unzip camus.zip -d camus_dataset


/kaggle/input/camus-human-heart-data/download: Zip archive data, at least v2.0 to extract, compression method=store
sample_data
mv: cannot stat 'download': No such file or directory
unzip:  cannot find or open camus.zip, camus.zip.zip or camus.zip.ZIP.


In [ ]:
!zip -r output_filename.zip /root/.cache/kagglehub/datasets

  adding: root/.cache/kagglehub/datasets/ (stored 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/ (stored 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/cardiacudc-dataset/ (stored 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/cardiacudc-dataset/versions/ (stored 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/cardiacudc-dataset/versions/1/ (stored 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/cardiacudc-dataset/versions/1/cardiacUDC_dataset.z03 (deflated 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/cardiacudc-dataset/versions/1/cardiacUDC_dataset.z05 (deflated 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/cardiacudc-dataset/versions/1/cardiacUDC_dataset.z02 (deflated 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/cardiacudc-dataset/versions/1/cardiacUDC_dataset.change2zip (deflated 0%)
  adding: root/.cache/kagglehub/datasets/xiaoweixumedicalai/ca

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp output_filename.zip /content/drive/MyDrive/


Mounted at /content/drive


In [ ]:
import urllib.parse
import IPython.display as display

file_path = '/content/output_filename.zip'
file_name = file_path.split('/')[-1]

# Encode the path for the link
encoded_path = urllib.parse.quote(file_path)

# This is the actual temporary download URL Colab uses
url = f'/download/{encoded_path}'

# Show it as a clickable link
display.HTML(f'<a href="{url}" target="_blank">Click here for direct download</a>')


In [ ]:
!wget -r -N -c -np --user mehran8 --ask-password https://physionet.org/files/mimic-iv-echo/0.1/


Password for user ‘mehran8’: 
--2025-08-16 11:21:27--  https://physionet.org/files/mimic-iv-echo/0.1/
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="PhysioNet", charset="UTF-8"
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 403 Forbidden
2025-08-16 11:21:27 ERROR 403: Forbidden.



# HVC

Downloading the data

In [1]:
import requests
import zipfile
import os

def download_file(url, extract_to="."):
    filename = url.split("/")[-1]  # get filename from URL

    print(f"Downloading {filename}...")
    response = requests.get(url)
    response.raise_for_status()  # ensure download was successful

    # Save the file
    with open(filename, "wb") as f:
        f.write(response.content)

    # If it's a ZIP, extract it
    if filename.endswith(".zip"):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted to {extract_to}")
    else:
        print(f"Saved {filename}")

# Example usage (replace with your actual URLs)
url1 = "https://data.unityimaging.net/files/2020-12-05/png-cache.zip"
url2 = "https://data.unityimaging.net/files/additional/view_index_20220705.csv"

download_file(url1, extract_to="data1")
download_file(url2, extract_to="data2")


Extracting png-cache.zip...
Extracted to data1
Saved view_index_20220705.csv


Preprocessing

In [ ]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm

root_dir = "/content/data1/png-cache/01"   # Unity dataset root
csv_file = "/content/view_index_20220705.csv"
output_dir = "dataset_cleaned"
os.makedirs(output_dir, exist_ok=True)

df = pd.read_csv(csv_file)
label_map = dict(zip(df['file'], df['view']))  # adjust if columns differ

target_size = (224, 224)

for fname, view in tqdm(label_map.items()):
    # locate file in nested folders
    sub1, sub2 = fname[3:5], fname[5:7]
    fpath = os.path.join(root_dir, sub1, sub2, fname)
    if not os.path.exists(fpath):
        print(f"missing: {fpath}")
        continue  # skip if file missing

    # open
    img = Image.open(fpath).convert("RGB")
    w, h = img.size

    # dynamic crop (50 top/bottom, 100 left/right)
    left, top, right, bottom = 100, 50, w-100, h-50
    img = img.crop((left, top, right, bottom))

    # resize
    img = img.resize(target_size)

    # save in clean folder
    outdir = os.path.join(output_dir, view)
    os.makedirs(outdir, exist_ok=True)
    img.save(os.path.join(outdir, fname))


Splitting the dataset

In [ ]:
import os
import shutil
import random

# --- Paths ---
dataset_dir = "dataset_cleaned"   # current dataset (all images per class)
output_dir = "dataset_split"      # new split dataset

# --- Split ratios ---
train_ratio, val_ratio, test_ratio = 0.7, 0.15, 0.15

# --- Fix random seed for reproducibility ---
random.seed(42)

# --- Create output structure ---
for split in ["train", "val", "test"]:
    for cls in os.listdir(dataset_dir):
        os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)

# --- Split images ---
for cls in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, cls)
    if not os.path.isdir(class_dir):
        continue

    images = os.listdir(class_dir)
    random.shuffle(images)

    n_total = len(images)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)

    train_files = images[:n_train]
    val_files = images[n_train:n_train+n_val]
    test_files = images[n_train+n_val:]

    # Move files
    for fname in train_files:
        shutil.copy(os.path.join(class_dir, fname), os.path.join(output_dir, "train", cls, fname))
    for fname in val_files:
        shutil.copy(os.path.join(class_dir, fname), os.path.join(output_dir, "val", cls, fname))
    for fname in test_files:
        shutil.copy(os.path.join(class_dir, fname), os.path.join(output_dir, "test", cls, fname))

    print(f"{cls}: {n_total} → train={len(train_files)}, val={len(val_files)}, test={len(test_files)}")

print("Dataset split completed at:", output_dir)
